In [1]:
#Importing the essential libraries
import requests
import nltk

import numpy as np 
import pandas as pd 

from IPython.display import display

nltk.download('omw-1.4')

from textblob import TextBlob

import csv

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import datetime
from datetime import datetime

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vrushtikhajanchi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
pd.set_option('display.max_rows', 10)

## Data Cleaning

In [ ]:
# For news collected from Google finance:

df1 = pd.read_excel('1.xlsx')
#delete time stamp in the current_time column
df1['current_date'] = df1['Current_Time'].str.slice(0,10)
#generagte multiplier value
df1['value'] = df1[df1["Publish_Date"].str.contains("ago")].Publish_Date.str.slice(0,1)
df1['value'] = df1['value'].replace(np.nan, 0)
#Converting this column to datetime format
df1['current_date'] = pd.to_datetime(df1['current_date'])
# convert "value" from String to int
df1 = df1.astype({'value':'int'})
df1['publish_date_formatted'] = df1['current_date'] \
                                - df1['days'] * df1['value'] * np.timedelta64(1, 'D') \
                                - df1['weeks'] * df1['value'] * np.timedelta64(1, 'W') \
                                - df1['months'] * df1['value'] * np.timedelta64(1, 'M')
def mdy_to_ymd(d):
    return datetime.strptime(d, '%b %d, %Y').strftime('%Y-%m-%d')
mask = df1['value'] == 0
df1.loc[mask, 'publish_date_formatted'] = df1.loc[mask, 'Publish_Date'].apply(mdy_to_ymd)
df1['publish_date_formatted'] = pd.to_datetime(df1['publish_date_formatted'])
#delete time stamp in the publish_date_formatted column
df1['publish_date_formatted'] = pd.to_datetime(df1['publish_date_formatted']).dt.date
df1.to_excel('MasterFile6.0.xlsx',index=False,header=True)

In [ ]:
# For news collected from Microsoft Bing:

df1 = pd.read_excel('Databricks_bing.xlsx')
df1['Publish_Date'] = df1['Publish_Date'].replace(np.nan, "-")
#generating new indicator column for date that's in "x d"/"y mon"

df1['days'] = df1['Publish_Date'].str.contains('d')
df1['days'] = df1['days'].map({True: 1, False: 0})

df1['months'] = df1['Publish_Date'].str.contains('mon')
df1['months'] = df1['months'].map({True: 1, False: 0})

#generagte multiplier value
df1['d value'] = df1[df1["Publish_Date"].str.contains("d")].Publish_Date.str.slice(0,-1)

df1['mon value'] = df1[df1["Publish_Date"].str.contains("mon")].Publish_Date.str.slice(0,-3)

df1['d value'] = df1['d value'].replace(np.nan, 0)
df1['mon value'] = df1['mon value'].replace(np.nan, 0)

#delete time stamp in the current_time column
df1['current_date'] = df1['Current_Time'].str.slice(0,10)

#Converting this column to datetime format
df1['current_date'] = pd.to_datetime(df1['current_date'])
# convert "d value" & "mon value" from String to int
df1 = df1.astype({'d value':'int'})
df1 = df1.astype({'mon value':'int'})

# Manipulating weird weirdo format
df1['publish_date_formatted'] = df1['current_date'] \
                                - df1['days'] * df1['d value'] * np.timedelta64(1, 'D') \
                                - df1['months'] * df1['mon value'] * np.timedelta64(1, 'M')
df1['publish_date_formatted'] = pd.to_datetime(df1['publish_date_formatted'])
#delete time stamp in the publish_date_formatted column
df1['publish_date_formatted'] = pd.to_datetime(df1['publish_date_formatted']).dt.date

# Keep publish_date_formatted column NaN for those news that don't have a publish date

df1.loc[df1['Publish_Date'] == "-", ['publish_date_formatted']] = "-"

### Removing irrelevant news

In [ ]:
df = pd.read_excel('MasterFile6.0.xlsx', sheet_name="Asana")
df = df[df["Title"].str.contains("Asanas")==False]
df = df[df["Title"].str.contains("asanas")==False]
df = df[df["Title"].str.contains("Yoga")==False]
df = df[df["Title"].str.contains("yoga")==False]
df.to_excel('asana.xlsx',index=False,header=True)

In [3]:
Masterfile = pd.ExcelFile('Copy_of_MasterFile7.0.xlsx')

In [4]:
company_name = ['Databricks', 'Confluent', 'Asana', 'Airtable', 'Reltio', 'Netskope', 'SkyHigh_Security',\
                'Domino_Data_Lab', 'Mapbox']

In [5]:
df = pd.read_excel(Masterfile, 'Databricks')

### Creating functions for each step

In [6]:
#Data cleaning
def preprocess(df):
    df.drop(columns=['Publish_Date','Current_Time', 'days', 'weeks', 'months', 'current_date', 'value', 'Picture'],\
                   inplace=True)
    df.columns = ['Headline', 'URL', 'Publisher', 'Summary', 'Date', 'Company_Name']
    #print(df.isna().sum())
    df.dropna(inplace=True)
    return df

In [7]:
#Sentence tokenization
def get_headlines(df):
    sentences=[]
    for h in df.Headline:
        if type(h) != str:
            h=str(h)
        sentences.append(h)
    return sentences

In [8]:
#Sentiment Analysis
def sentiment_analysis(sent):
    textblob_sentiment=[]
    for s in sent:
        txt= TextBlob(s)
        a= txt.sentiment.polarity
        b= txt.sentiment.subjectivity
        textblob_sentiment.append([s,a,b])
        df_textblob = pd.DataFrame(textblob_sentiment, columns =['Sentence', 'Polarity', 'Subjectivity'])
    return df_textblob

In [9]:
#Sentiment categorization
def sentiment_type(df):
    print('News with positive sentiment:', len(df[df.Polarity>0]))
    print('News with negative sentiment:', len(df[df.Polarity<0]))
    print('News with neutral sentiment:', len(df[df.Polarity==0]))

In [10]:
def after_process(df):
    df['Sentiment'] = df['Polarity'].apply(lambda x: 'Negative' if (x<0) \
                            else 'Positive' if (x>0) \
                            else 'Neutral')
    df['Sentiment Score'] = df['Polarity'] * 100
    df['Sentiment Score']  = round(df['Sentiment Score'], 2)
    df = pd.get_dummies(df, columns=['Sentiment'])
    df.rename(columns={'Sentiment_Negative':'Negative', 'Sentiment_Positive':'Positive',\
                            'Sentiment_Neutral':'Neutral'}, inplace=True)
    return df

In [11]:
#Percentile threshold method
def percentile_scores(df, pos_perc, neg_perc):
#     df_pos = df[df['Sentiment Score'] >0]
#     df_neg = df[df['Sentiment Score'] <0]
    
    pos_score = df[df['Sentiment Score']>0]['Sentiment Score']
    neg_score = df[df['Sentiment Score']<0]['Sentiment Score']
    
    pos_thresh = np.percentile(pos_score, pos_perc)
    neg_thresh = np.percentile(neg_score, neg_perc)

    print(str(pos_perc)+ " percentile for positive sentiment score:", pos_thresh)
    print(str(neg_perc)+ " percentile for negative sentiment score:", neg_thresh)
    
    df = df[(df['Sentiment Score']<=neg_thresh) | (df['Sentiment Score']>=pos_thresh)]
    return df

### Code for Word_Cloud

In [12]:
def high_impact_words(df):
    df_high_impact_words = pd.DataFrame(columns = df.columns)

    # Stores the high impact words in a list
    techwords_list = []

    # Read high impact words from CSV file
    with open('AWS_tech_list.csv', 'r') as file:
        reader = csv.reader(file)
        for row in reader:
            word = row[0].lower()
            techwords_list.append(word)

    # The tech word list might have dupliacte words. This can cause the word cloud
    # to falsely grow. To resolve this, we need to remove the duplicate words from
    # the word cloud.
    techwords_list = [*set(techwords_list)]
    
    # Stores the list of words which are hit
    techwords_hit = []

    for index, row in df.iterrows():
        #print(row['Date'], row['Headline'])
        row['Word_Hit'] = ""
        df_high_impact_words = df_high_impact_words.append(row, 
                                                   ignore_index = True)
        for hit_word in techwords_list:
            # Use below if we need more hits, like debut and debuts. But that
            # also matches words like AI in HAIL.
            # if hit_word in row['Headline'].lower():
            if hit_word in row['Headline'].lower().split():
                techwords_hit.append(hit_word)
                row['Word_Hit'] = hit_word
                df_high_impact_words = df_high_impact_words.append(row, 
                                                   ignore_index = True)
                # print(hit_word + " --> " + row['Headline'])
   
    print("Number of headlines : " + str(df_high_impact_words['Headline'].count()))
    # print(techwords_hit)
    print("Tech words hit : " + str(len(techwords_hit)))

    return df_high_impact_words

## Sentiment analysis

In [13]:
# create a dictionary to store DataFrames for each company
# company_dfs = {}
tech_word_dfs = {}

company_name
# iterate over each company and read its data into a DataFramefor x in company_name:
for x in company_name:
    print(x)
    #print("\n")
    df = pd.read_excel(Masterfile, x)
    company_list = [x] * len(df)  # create list of company names with same length as dataframe
    df['Company_Name'] = company_list  # add new column with company name
    
    df = preprocess(df)
    
    sentences = get_headlines(df)
    
    sentiment_df = sentiment_analysis(sentences)

    sentiment_df2 = after_process(sentiment_df)
    
    #new code to remove Nans
    df.reset_index(inplace=True)
    df.drop(columns='index', inplace=True)
    sentiment_df2['Date']= df['Date']
    sentiment_df2['URL']= df.URL
    sentiment_df2['Headline'] = df.Headline
    sentiment_df2['Company_Name'] = df['Company_Name'] # add company name to the new DataFrame

    sentiment_df2['Sentiment_type']=sentiment_df2.Polarity.apply(lambda x: 'Positive' if x>0 else 'Negative' if x<0 \
                                                       else 'Neutral')     
    sentiment_df2.rename(columns={'Sentence':'Sentiment headline'}, inplace=True)
    
    sentiment_df2=percentile_scores(sentiment_df2, 75, 25)
    
    # Create high impact word hit list
    tech_word_df = high_impact_words(sentiment_df2)
    tech_word_dfs[x] = tech_word_df
        
# write DataFrames for each company to separate sheets in the same Excel file
with pd.ExcelWriter('Company_TechWordCloud_2_OG.xlsx') as writer:
    for x, df in tech_word_dfs.items():
        df.to_excel(writer, sheet_name=x, index=False)

Databricks
75 percentile for positive sentiment score: 40.0
25 percentile for negative sentiment score: -23.75
Number of headlines : 52
Tech words hit : 24
Confluent
75 percentile for positive sentiment score: 37.5
25 percentile for negative sentiment score: -29.3775
Number of headlines : 58
Tech words hit : 24
Asana
75 percentile for positive sentiment score: 50.0
25 percentile for negative sentiment score: -27.5
Number of headlines : 73
Tech words hit : 20
Airtable
75 percentile for positive sentiment score: 50.0
25 percentile for negative sentiment score: -22.5
Number of headlines : 42
Tech words hit : 8
Reltio
75 percentile for positive sentiment score: 40.0
25 percentile for negative sentiment score: -17.78
Number of headlines : 15
Tech words hit : 4
Netskope
75 percentile for positive sentiment score: 50.0
25 percentile for negative sentiment score: -17.5
Number of headlines : 35
Tech words hit : 13
SkyHigh_Security
75 percentile for positive sentiment score: 80.0
25 percentile f